# Collecting data

In [11]:
import torch
import pennylane as qml
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [12]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [20]:
from torch.utils.data import DataLoader

loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=0,
                                          pin_memory=True),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=0,
                                          pin_memory=True),
}

In [21]:
def test(model):
    # Test the model
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for i, (images, labels) in enumerate(loaders['test']):
            print(i)
            images, labels = images.to(device), labels.to(device)
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1]
            accuracy += (pred_y == labels).sum().item() 
        accuracy /= len(loaders['test'].dataset)
        print('Test Accuracy of the model on the 10000 test images: {:.2f} %'.format(accuracy*100.0))

# Loading model

In [22]:
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(features=inputs, wires=range(n_qubits))
    
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    return [qml.expval(qml.PauliY(wires=i)) for i in range(n_qubits)]

In [23]:
n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

In [24]:
import torch.nn as nn

class HybridNN(nn.Module):
    def __init__(self):
        super(HybridNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=16,              
                out_channels=32,            
                kernel_size=5,              
                stride=1,                   
                padding=2,    
            ),     
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),                
        )
        self.fc_1 = nn.Linear(32 * 7 * 7, 24)
        
        # LIST USAGE?
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_2 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_3 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_4 = qml.qnn.TorchLayer(qnode, weight_shapes)
        
        self.qlayer_1.to(device)
        self.qlayer_2.to(device)
        self.qlayer_3.to(device)
        self.qlayer_4.to(device)
        
        self.fc_2 = nn.Linear(24, 10)

        
    def forward(self, x):

        x = self.conv1(x)

        x = self.conv2(x)

        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1) 

        x = self.fc_1(x)
        #print('Before split')
        x_1, x_2, x_3, x_4 = torch.split(x, 6, dim=1) # second argument is number of elements in one new tensor
        #print('After split')
        #x = torch.Tensor(0)
        
        x_1 = self.qlayer_1(x_1)
        x_2 = self.qlayer_2(x_2)
        x_3 = self.qlayer_3(x_3)
        x_4 = self.qlayer_4(x_4)
        
        #print(x.device)
        
        x = torch.cat([x_1, x_2, x_3, x_4], axis=1)
        x = x.to(device)
        
        logits = self.fc_2(x)
        
        return logits

In [25]:
hnn = HybridNN()
hnn = hnn.to(device)
hnn.load_state_dict(torch.load('trained_model_24qubits_3_layers_with_strong_entagling.pt'))
hnn.eval()
print(hnn)

HybridNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_1): Linear(in_features=1568, out_features=24, bias=True)
  (qlayer_1): <Quantum Torch Layer: func=qnode>
  (qlayer_2): <Quantum Torch Layer: func=qnode>
  (qlayer_3): <Quantum Torch Layer: func=qnode>
  (qlayer_4): <Quantum Torch Layer: func=qnode>
  (fc_2): Linear(in_features=24, out_features=10, bias=True)
)


In [26]:
test(hnn)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Test Accuracy of the model on the 10000 test images: 97.41 %


In [27]:
sample = next(iter(loaders['test']))
images, labels = sample
actual_number = labels[:10].numpy()
actual_number

array([7, 4, 9, 7, 2, 1, 8, 0, 0, 9], dtype=int64)

In [28]:
test_output = hnn(images[:10].to(device))
pred_y = torch.max(test_output, 1)[1].cpu().numpy()
print(f'Prediction number: {pred_y}')
print(f'Actual number: {actual_number}')

Prediction number: [7 4 9 7 2 1 8 0 0 9]
Actual number: [7 4 9 7 2 1 8 0 0 9]
